# A2DI - TP6

## Exercice 2 - Discriminatif vs Génératif

(Question 1)

In [3]:
from sklearn import datasets
from sklearn import linear_model
import matplotlib.pyplot as plt
import numpy as np
import scipy

# Permet l'affichage des graphiques
%matplotlib inline

data = scipy.io.loadmat('20news_w100.mat')
X = data['documents'].toarray()
c = data["newsgroups"][0]-1

n = np.shape(c)[0]
d = np.shape(X)[0]
n_class = len(np.bincount(c))

# X_input : ensemble X d'exemple
# c_input : labels de X
# k       : nombre de pli
# n_class : nombre de class
def kfold_data(X_input, c_input, k, nb_class):
    # On transpose pour que ce soit plus pratique
    x_transpose = np.array(X_input).T

    # On commence par regrouper chaque exemple
    # par leurs labels respectifs
    x_by_labels = [[] for _ in range(nb_class)]
    for i in range(len(c_input)):
        x_by_labels[c_input[i]].append(x_transpose[i])
        np.random.shuffle(x_by_labels[c_input[i]])
    
    # On crée les différents plis selon le nombre de 
    # chacune des classes
    size_pli_by_class = [0 for _ in range(nb_class)]
    rest_pli_by_class = [0 for _ in range(nb_class)]
    
    for i in range(nb_class):
        size_pli_by_class[i] = len(x_by_labels[i]) // k
        rest_pli_by_class[i] = len(x_by_labels[i]) % k
    
    k_plis = []
    
    # Chaque boucle crée un nouveau pli
    for i in range(k):
        xx = []
        yy = []

        for j in range(nb_class):
            xx += x_by_labels[j][i*size_pli_by_class[j]:(i+1)*size_pli_by_class[j]]
            yy += [j for _ in range(size_pli_by_class[j])]
        
        s = np.random.get_state()
        np.random.shuffle(xx)
        np.random.set_state(s)
        np.random.shuffle(yy)
        
        k_plis.append([xx, yy])
    
    # On ajoute le reste de chaque classe au dernier pli
    for j in range(nb_class):
        k_plis[-1][0] += x_by_labels[j][k*size_pli_by_class[j]:]
        k_plis[-1][1] += [j for _ in range(rest_pli_by_class[j])]
        
    # Pour finir on crée toutes les possibilités de répartition des
    # plis
    plis = [[] for _ in range(4)]
    
    for i in range(len(k_plis)):
        xx = []
        yy = []
        for j in range(len(k_plis)):
            if j != i:
                xx += k_plis[j][0]
                yy += k_plis[j][1]
        plis[0].append(np.array(xx).T)
        plis[1].append(np.array(yy))
        plis[2].append(np.array(k_plis[i][0]).T)
        plis[3].append(np.array(k_plis[i][1]))
        
    return plis[0], plis[1], plis[2], plis[3]

k_plis = kfold_data(X, c, 5, n_class)

(Question 2)

In [4]:
print(np.shape(k_plis))
print(np.shape(k_plis[0][0]))

print(np.bincount(k_plis[1][0]))
print(np.bincount(k_plis[3][0]))

print(np.bincount(k_plis[1][4]))
print(np.bincount(k_plis[3][4]))

for p in range(len(k_plis[0])):
    # On l'entraine
    logreg = linear_model.LogisticRegression(C=1e5, multi_class='multinomial', solver='newton-cg')
    logreg.fit(np.transpose(k_plis[0][p]), k_plis[1][p])
    
    err = 0
    # On test sur tous les exemples
    X_ex = np.transpose(k_plis[2][p])
    pred = logreg.predict(X_ex)

    for i in range(len(X_ex)):
        if pred[i] != k_plis[3][p][i]:
            err += 1
    err = (err / len(X_ex)) * 100
    print(err)

(4, 5)
(100, 12995)
[3684 2816 2126 4369]
[ 921  703  531 1092]
[3684 2812 2124 4368]
[ 921  707  533 1093]
18.94056051740068
19.83369263935941
19.49491838620265
18.878965198644902
17.670559311616472
